In [51]:
# Импорт необходимых библиотек
import requests
import json
import time
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import uuid

# Базовый URL нашего API
BASE_URL = "http://localhost:8000"  # Измените на фактический URL, если сервис запущен не локально


In [52]:
# 1. Тестирование регистрации пользователя
def test_user_registration():
    global token
    
    # Создаем уникальное имя пользователя
    username = f"testuser_{int(time.time())}"
    
    # Данные для регистрации
    user_data = {
        "username": username,
        "email": f"{username}@example.com",
        "password": "password123"
    }
    
    # Отправляем запрос на регистрацию
    response = requests.post(f"{BASE_URL}/users/", json=user_data)
    display_response(response, "1. Регистрация пользователя")
    
    # Запрашиваем токен для аутентификации
    token_data = {
        "username": username,
        "password": "password123"
    }
    token_response = requests.post(
        f"{BASE_URL}/token", 
        data=token_data,
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    display_response(token_response, "Получение токена")
    
    # Сохраняем токен для последующих запросов
    if token_response.status_code == 200:
        token = token_response.json()["access_token"]
        print(f"Токен получен и сохранен для дальнейшего использования")
    
    return username

# Запускаем тест регистрации
test_username = test_user_registration()

Status Code: 200


,username,email,id,created_at
0,testuser_1742942074,testuser_1742942074@example.com,9,2025-03-25T22:34:34.309614+00:00


Status Code: 200


,access_token,token_type
0,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJzdWIiO...,bearer


Токен получен и сохранен для дальнейшего использования


In [53]:
# 1. Тестирование регистрации пользователя
def test_user_registration():
    global token
    
    # Создаем уникальное имя пользователя
    username = f"testuser_{int(time.time())}"
    
    # Данные для регистрации
    user_data = {
        "username": username,
        "email": f"{username}@example.com",
        "password": "password123"
    }
    
    # Отправляем запрос на регистрацию
    response = requests.post(f"{BASE_URL}/users/", json=user_data)
    display_response(response, "1. Регистрация пользователя")
    
    # Запрашиваем токен для аутентификации
    token_data = {
        "username": username,
        "password": "password123"
    }
    token_response = requests.post(
        f"{BASE_URL}/token", 
        data=token_data,
        headers={"Content-Type": "application/x-www-form-urlencoded"}
    )
    display_response(token_response, "Получение токена")
    
    # Сохраняем токен для последующих запросов
    if token_response.status_code == 200:
        token = token_response.json()["access_token"]
        print(f"Токен получен и сохранен для дальнейшего использования")
    
    return username


# 2. Тестирование создания короткой ссылки
def test_create_short_link():
    # Проверяем, есть ли токен
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return
    
    # Создаем уникальный alias
    unique_alias = f"example_{uuid.uuid4().hex[:8]}"
    
    # Данные для создания короткой ссылки
    link_data = {
        "original_url": "https://www.example.com/very/long/url/that/needs/shortening",
        "custom_alias": unique_alias
    }
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Отправляем запрос на создание короткой ссылки
    response = requests.post(f"{BASE_URL}/links/shorten", json=link_data, headers=headers)
    display_response(response, f"2. Создание короткой ссылки с кастомным alias '{unique_alias}'")
    
    # Получаем короткий код из ответа
    if response.status_code == 200:
        short_code = response.json()["short_code"]
        return short_code

# 3. Создание ссылки с временем жизни
def test_create_link_with_expiry():
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return
    
    # Устанавливаем время жизни ссылки (например, 1 час от текущего времени)
    # Используем правильный формат ISO 8601
    expiry_time = (datetime.now() + timedelta(hours=1)).isoformat()
    
    # Данные для создания короткой ссылки с временем жизни
    link_data = {
        "original_url": "https://www.example.com/temporary/link",
        "expires_at": expiry_time
    }
    
    # Выводим данные для отладки
    print(f"Отправляемые данные: {json.dumps(link_data, default=str)}")
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Отправляем запрос на создание короткой ссылки с временем жизни
    response = requests.post(f"{BASE_URL}/links/shorten", json=link_data, headers=headers)
    display_response(response, "3. Создание ссылки с временем жизни")
    
    # Получаем короткий код из ответа
    if response.status_code == 200:
        short_code = response.json()["short_code"]
        return short_code
    else:
        # Выводим более подробную информацию об ошибке
        print(f"Ошибка создания ссылки с временем жизни: {response.text}")
        return None


# 4. Тестирование получения информации о ссылке
def test_get_link_info(short_code):
    if not short_code:
        print("Короткий код не получен. Пожалуйста, сначала создайте ссылку.")
        return
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Отправляем запрос на получение информации о ссылке
    response = requests.get(f"{BASE_URL}/links/{short_code}", headers=headers)
    display_response(response, f"4. Получение информации о ссылке {short_code}")
    return response.status_code == 200

# 5. Тестирование получения статистики по ссылке
def test_get_link_stats(short_code):
    if not short_code:
        print("Короткий код не получен. Пожалуйста, сначала создайте ссылку.")
        return
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Отправляем запрос на получение статистики по ссылке
    response = requests.get(f"{BASE_URL}/links/{short_code}/stats", headers=headers)
    display_response(response, f"5. Получение статистики по ссылке {short_code}")
    return response.status_code == 200

# 6. Тестирование перенаправления по короткой ссылке
def test_redirect(short_code):
    if not short_code:
        print("Короткий код не получен. Пожалуйста, сначала создайте ссылку.")
        return
    
    # Отправляем запрос на перенаправление (с параметром allow_redirects=False, чтобы увидеть ответ с перенаправлением)
    response = requests.get(f"{BASE_URL}/{short_code}", allow_redirects=False)
    
    # Выводим информацию о перенаправлении
    print(f"6. Перенаправление по короткой ссылке {short_code}")
    print(f"Status Code: {response.status_code}")
    print(f"Location: {response.headers.get('Location', 'No redirect location')}")
    
    # Проверяем, что статус-код соответствует перенаправлению
    if 300 <= response.status_code < 400:
        print("Успешное перенаправление!")
        return True
    else:
        print("Ошибка перенаправления.")
        return False

# 7. Тестирование обновления ссылки
def test_update_link(short_code):
    if not short_code or not token:
        print("Короткий код или токен не получены. Пожалуйста, сначала создайте ссылку и авторизуйтесь.")
        return
    
    # Данные для обновления ссылки
    update_data = {
        "original_url": "https://www.example.com/updated/url"
    }
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Отправляем запрос на обновление ссылки
    response = requests.put(f"{BASE_URL}/links/{short_code}", json=update_data, headers=headers)
    display_response(response, f"7. Обновление ссылки {short_code}")
    
    # Проверяем, что ссылка обновилась
    success = response.status_code == 200
    if success:
        # Проверим информацию об обновленной ссылке
        test_get_link_info(short_code)
    
    return success

# 8. Тестирование удаления ссылки
def test_delete_link(short_code):
    if not short_code or not token:
        print("Короткий код или токен не получены. Пожалуйста, сначала создайте ссылку и авторизуйтесь.")
        return
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Отправляем запрос на удаление ссылки
    response = requests.delete(f"{BASE_URL}/links/{short_code}", headers=headers)
    
    # Выводим результат
    print(f"8. Удаление ссылки {short_code}")
    print(f"Status Code: {response.status_code}")
    if response.status_code == 204:
        print("Ссылка успешно удалена!")
        success = True
    else:
        print(f"Ошибка при удалении ссылки: {response.text}")
        success = False
    
    # Проверяем, что ссылка действительно удалена
    check_response = requests.get(f"{BASE_URL}/links/{short_code}", headers=headers)
    if check_response.status_code == 404:
        print("Подтверждено: ссылка не найдена после удаления.")
    else:
        print(f"Внимание: ссылка все еще доступна после удаления! Статус: {check_response.status_code}")
    
    return success

# 9. Тестирование поиска ссылки по оригинальному URL
# Исправленный тест поиска ссылки по оригинальному URL
# Обновленный тест поиска ссылки по оригинальному URL
def test_search_by_original_url():
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return False
    
    # Создаем уникальную ссылку для поиска
    unique_url = f"https://www.example.com/unique/url/{uuid.uuid4().hex}"
    
    # Данные для создания ссылки
    link_data = {
        "original_url": unique_url
    }
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Сначала создаем ссылку
    create_response = requests.post(f"{BASE_URL}/links/shorten", json=link_data, headers=headers)
    display_response(create_response, "Создание ссылки для поиска")
    
    if create_response.status_code != 200:
        print("Не удалось создать ссылку для поиска.")
        return False
    
    # Даем немного времени на обработку
    time.sleep(1)
    
    # Используем новый эндпоинт для поиска
    search_response = requests.get(f"{BASE_URL}/search-url?url={unique_url}", headers=headers)
    display_response(search_response, "9. Поиск ссылки по оригинальному URL (новый эндпоинт)")
    
    return search_response.status_code == 200


# 10. Тестирование получения истории истекших ссылок
# Обновленный тест получения истории истекших ссылок
def test_get_expired_links():
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return False
    
    # Создаем ссылку с очень коротким временем жизни (2 секунды)
    expiry_time = (datetime.now() + timedelta(seconds=2)).isoformat()
    
    link_data = {
        "original_url": "https://example.com/expires-very-soon",
        "expires_at": expiry_time
    }
    
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Создаем ссылку
    create_response = requests.post(f"{BASE_URL}/links/shorten", json=link_data, headers=headers)
    display_response(create_response, "Создание ссылки с очень коротким временем жизни")
    
    if create_response.status_code != 200:
        print("Не удалось создать ссылку с временем жизни.")
        return False
    
    # Ждем, пока истечет срок действия ссылки
    print("Ожидаем истечения срока действия ссылки (3 секунды)...")
    time.sleep(3)
    
    # Используем новый эндпоинт для получения истории истекших ссылок
    expired_response = requests.get(f"{BASE_URL}/expired-links", headers=headers)
    display_response(expired_response, "10. Получение истории истекших ссылок (новый эндпоинт)")
    
    return expired_response.status_code == 200 and len(expired_response.json()) > 0

# 11. Тестирование настройки автоматического удаления неиспользуемых ссылок
def test_cleanup_unused_links():
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    # Отправляем запрос на настройку очистки
    response = requests.post(f"{BASE_URL}/links/cleanup?days=30", headers=headers)
    display_response(response, "11. Настройка автоматического удаления неиспользуемых ссылок")
    
    return response.status_code == 200

# Для теста удаления создадим новую ссылку
def create_link_for_deletion():
    if not token:
        print("Токен не получен. Пожалуйста, сначала выполните регистрацию.")
        return
    
    # Данные для создания короткой ссылки
    link_data = {
        "original_url": "https://www.example.com/link/to/be/deleted",
        "custom_alias": f"delete_{uuid.uuid4().hex[:8]}"
    }
    
    # Заголовки с токеном авторизации
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    
    # Отправляем запрос на создание короткой ссылки
    response = requests.post(f"{BASE_URL}/links/shorten", json=link_data, headers=headers)
    display_response(response, "Создание ссылки для удаления")
    
    # Получаем короткий код из ответа
    if response.status_code == 200:
        short_code = response.json()["short_code"]
        return short_code
    return None




In [54]:
# Функция для запуска всех тестов
def run_all_tests():
    results = []
    
    # Тестируем получение информации и статистики
    print("\n=== Тестирование получения информации и статистики ===")
    if short_code1:
        info_success = test_get_link_info(short_code1)
        stats_success = test_get_link_stats(short_code1)
        results.append({"Тест": "Получение информации о ссылке", "Результат": "✅ Успешно" if info_success else "❌ Ошибка"})
        results.append({"Тест": "Получение статистики по ссылке", "Результат": "✅ Успешно" if stats_success else "❌ Ошибка"})
    
    # Тестируем перенаправление
    print("\n=== Тестирование перенаправления ===")
    if short_code1:
        redirect_success = test_redirect(short_code1)
        results.append({"Тест": "Перенаправление по короткой ссылке", "Результат": "✅ Успешно" if redirect_success else "❌ Ошибка"})
    
    # Тестируем обновление ссылки
    print("\n=== Тестирование обновления ссылки ===")
    if short_code1:
        update_success = test_update_link(short_code1)
        results.append({"Тест": "Обновление ссылки", "Результат": "✅ Успешно" if update_success else "❌ Ошибка"})
    
    # Тестируем поиск ссылки по оригинальному URL
    print("\n=== Тестирование поиска по оригинальному URL ===")
    search_success = test_search_by_original_url()
    results.append({"Тест": "Поиск ссылки по оригинальному URL", "Результат": "✅ Успешно" if search_success else "❌ Ошибка"})
    
    # Тестируем получение истории истекших ссылок
    print("\n=== Тестирование получения истории истекших ссылок ===")
    expired_success = test_get_expired_links()
    results.append({"Тест": "Получение истории истекших ссылок", "Результат": "✅ Успешно" if expired_success else "❌ Ошибка"})
    
    # Тестируем настройку автоматического удаления
    print("\n=== Тестирование настройки автоматического удаления ===")
    cleanup_success = test_cleanup_unused_links()
    results.append({"Тест": "Настройка автоудаления неиспользуемых ссылок", "Результат": "✅ Успешно" if cleanup_success else "❌ Ошибка"})
    
    # Тестируем удаление ссылки
    print("\n=== Тестирование удаления ссылки ===")
    delete_code = create_link_for_deletion()
    if delete_code:
        delete_success = test_delete_link(delete_code)
        results.append({"Тест": "Удаление ссылки", "Результат": "✅ Успешно" if delete_success else "❌ Ошибка"})
    
    # Выводим итоговую таблицу результатов
    print("\n=== ИТОГОВЫЕ РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ ===")
    results_df = pd.DataFrame(results)
    display(results_df)
    
    # Подсчитываем общий результат
    success_count = results_df[results_df["Результат"].str.contains("✅")].shape[0]
    total_count = results_df.shape[0]
    success_rate = (success_count / total_count) * 100
    
    print(f"\nУспешно выполнено {success_count} из {total_count} тестов ({success_rate:.2f}%)")
    
    if success_rate == 100:
        print("🎉 Все тесты успешно пройдены! API полностью функционален.")
    elif success_rate >= 80:
        print("🔶 Большинство тестов пройдено успешно. Некоторые функции требуют доработки.")
    else:
        print("❌ Значительное количество тестов не пройдено. Требуется серьезная доработка API.")

# Запускаем все тесты
run_all_tests()


=== Тестирование получения информации и статистики ===


Status Code: 200


,short_code,original_url,created_at,clicks,last_used,expires_at,owner_id
0,example_8c449266,https://www.example.com/updated/url,2025-03-25T22:21:00.658219+00:00,1,2025-03-25T22:21:01.431557+00:00,None,8


Status Code: 200


,short_code,original_url,created_at,clicks,last_used,expires_at,owner_id
0,example_8c449266,https://www.example.com/updated/url,2025-03-25T22:21:00.658219+00:00,1,2025-03-25T22:21:01.431557+00:00,None,8


...

Status Code: 200


,short_code,original_url,created_at
0,delete_3f32c5fd,https://www.example.com/link/to/be/deleted,2025-03-25T22:34:46.986622+00:00


8. Удаление ссылки delete_3f32c5fd
Status Code: 204
Ссылка успешно удалена!
Подтверждено: ссылка не найдена после удаления.

=== ИТОГОВЫЕ РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ ===


,Тест,Результат
0,Получение информации о ссылке,✅ Успешно
1,Получение статистики по ссылке,✅ Успешно
2,Перенаправление по короткой ссылке,❌ Ошибка
3,Обновление ссылки,❌ Ошибка
4,Поиск ссылки по оригинальному URL,❌ Ошибка
5,Получение истории истекших ссылок,❌ Ошибка
6,Настройка автоудаления неиспользуемых ссылок,✅ Успешно
7,Удаление ссылки,✅ Успешно



Успешно выполнено 4 из 8 тестов (50.00%)
❌ Значительное количество тестов не пройдено. Требуется серьезная доработка API.


In [ ]:
docker-compose down
docker-compose up --build -d
